In [2]:
import cv2
import torch
import torch.nn as nn
import numpy as np
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
from torchvision import models
from PIL import Image

import pandas as pd

In [3]:
##1.Schritt einbringen von dem 3. Frame von MOT16-04 

img = Image.open("TestFrame000003.jpg")

In [4]:
##2.Schritt einbringen von x, y, w, h aus der gt Datei mit PANDAS

    ##gt.txt öffnen
open("gt.txt", "r")
Koordinaten = open("gt.txt", "r")

    ##gt.txt in PANDAS Tabelle
data = pd.read_csv('gt.txt', header = None)
data.columns = ['Frame', 'ID', 'x', 'y', 'w', 'h', 'Dies', 'Das', 'Visability']

    ##nur Zeilen die mit 3 starten - alle anderen löschen
data.drop(data[data.Frame != 3].index, inplace=True)

    ##nur x, y, w, h
data.drop(['Frame', 'ID', 'Dies', 'Das', 'Visability'], axis=1, inplace=True)
print(data)

x = data.at[2, 'x']
y = data.at[2, 'y']
w = data.at[2, 'w']
h = data.at[2, 'h']

           x    y    w    h
2       1362  568  103  241
1052     372  407   80  239
1732     102  549   83  250
2782    1732  457   76  212
3832    1104  978   78  210
...      ...  ...  ...  ...
88299    264   71   54  166
89349    209   95   41  179
90399    914 -107   38  115
98591   1217  -14   65  147
106177    39  309  115  117

[99 rows x 4 columns]


In [5]:
##3.Schritt ausschneiden einer Bounding Box von einem dieser Frames
img2 = img.crop((x, y, x+w, y+h))
img2.save('TestBoundingBox.png')

In [58]:
##4.Schritt Transformieren und Normalisieren des Bounding-Box Bildes

    ##Normalization: image = (image - mean) / std
        ##torchvision.transforms.Resize(size, interpolation=2)    size = (h, w)

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

trans_img = transform (img2)

trans_img.size()
trans_img_size = trans_img.unsqueeze(0)
trans_img_size.size()

torch.Size([1, 3, 598, 256])

In [59]:
##5.Schritt ResNet34 Model laden
cnn = torchvision.models.resnet34(pretrained=True)
cnn = torch.nn.Sequential(*(list(cnn.children())[:-1])) 

In [57]:
##6.Schritt Bounding Box Bild durch das CNN jagen
result = cnn(trans_img_size)
result.view(512)

tensor([0.8707, 0.8550, 1.0356, 0.9202, 0.8550, 1.0271, 1.0008, 0.8715, 0.9966,
        0.9982, 0.8885, 0.8526, 0.9885, 0.8443, 0.8160, 1.1151, 0.9773, 0.9675,
        0.9039, 0.8446, 1.3552, 0.8911, 1.0123, 0.9089, 0.9153, 0.9438, 0.9299,
        0.9428, 0.9977, 1.0652, 1.0114, 0.9382, 0.8584, 0.8747, 0.8892, 1.0681,
        0.9393, 0.8797, 0.9721, 0.8740, 0.8887, 0.9539, 0.9548, 0.9862, 0.8565,
        0.9653, 0.9207, 1.0210, 0.9118, 0.9027, 0.8447, 1.2383, 0.8582, 0.9673,
        0.8983, 0.9060, 0.8572, 0.7836, 0.9808, 0.9536, 0.9295, 1.0631, 0.9304,
        0.9447, 0.9278, 0.7992, 0.9094, 0.9617, 0.8389, 0.9482, 1.0417, 0.9746,
        0.9211, 0.9276, 0.9321, 1.2293, 0.8616, 0.9883, 1.5001, 0.9103, 0.9789,
        1.0419, 0.9160, 0.7732, 0.9130, 1.0819, 0.9976, 0.9110, 0.9224, 0.8120,
        1.3816, 0.8510, 0.9796, 0.9269, 0.9035, 1.0673, 1.0364, 1.0012, 0.9736,
        0.9298, 0.8890, 0.9535, 0.8620, 0.8529, 0.8957, 0.8433, 0.9875, 0.9692,
        0.9150, 0.8689, 1.0480, 0.9467, 